created by Dirk Derichsweiler, k8s@hpe.com, April 2020

**IMPORTANT:** select the right server

In [1]:
ls ~/k8sconfig

all                hcp_barem_sy480                   powerweek2-config
azure-ddk8s        hpecp_csi_demo_admin_kubeconfig   rancher-demo02-centos
azure-ddk8sctc2    hpecp-DDK8S-HPECP-admin-config    rancher-rke-centos
bigmama-k3s        hpecp-DD_PLAYGROUND-admin-config
ddk1-ddk8s-config  powerweek1-config


In [3]:
export KUBECONFIG=~/k8sconfig/powerweek1-config

error: the server doesn't have a resource type "nodesy"


: 1

# Powerweek - Containers in Action - Level 100

<img src="media/nodes.png" alt="SC" width="700" height="700" style="float:left">

In [4]:
kubectl get nodes

NAME   STATUS   ROLES    AGE    VERSION
ddk1   Ready    master   260d   v1.17.0
ddk2   Ready    <none>   117d   v1.17.0
ddk3   Ready    <none>   117d   v1.17.0
ddk4   Ready    <none>   117d   v1.17.0
ddk5   Ready    <none>   117d   v1.17.0
ddk6   Ready    <none>   117d   v1.17.0


In [5]:
kubectl describe node ddk1

Name:               ddk1
Roles:              master
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/os=linux
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=ddk1
                    kubernetes.io/os=linux
                    node-role.kubernetes.io/master=
                    rook-edgefs=cluster
Annotations:        csi.volume.kubernetes.io/nodeid:
                      {"csi.hpe.com":"00505684-e6f1-6464-6b31-000000000000","io.rancher.longhorn":"ddk1","rook-ceph.rbd.csi.ceph.com":"ddk1"}
                    kubeadm.alpha.kubernetes.io/cri-socket: /var/run/dockershim.sock
                    node.alpha.kubernetes.io/ttl: 0
                    projectcalico.org/IPv4Address: 10.0.31.73/24
                    projectcalico.org/IPv4IPIPTunnelAddr: 192.168.12.128
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Sun, 18 Aug 2019 15:57:13 +0200
Taints:            

<img src="media/namespaces.png" alt="SC" width="700" height="700" style="float:left">

In [6]:
kubectl get namespaces

NAME                   STATUS   AGE
backup                 Active   54d
ctc                    Active   260d
default                Active   260d
ingress-system         Active   97d
kube-node-lease        Active   260d
kube-public            Active   260d
kube-system            Active   260d
kubernetes-dashboard   Active   133d
longhorn-system        Active   96d
metallb-system         Active   260d
oneview                Active   221d
powerweek              Active   103m
sftp4ov                Active   212d
stephan                Active   214d
test-csi               Active   35d
velero                 Active   54d


In [7]:
kubectl delete namespace powerweek

namespace "powerweek" deleted


In [ ]:
kubectl describe namespaces powerweek

# connect to our external accesible cluster

In [93]:
export KUBECONFIG=~/k8sconfig/powerweek2-config

In [94]:
kubectl get all 

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.0.0.1     <none>        443/TCP   3d10h


<img src="media/container.png" alt="SC" width="350" height="350" style="float:left">
<img src="media/pod.png" alt="SC" width="700" height="700" style="float:left">
<img src="media/deployment.png" alt="SC" width="700" height="700" style="float:left">
<img src="media/pv.png" alt="SC" width="700" height="700" style="float:left">

In [95]:
cat << 'EOF' | kubectl apply -f -
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: pwsurvey-db
spec:
  selector:
    matchLabels:
      app: pwsurvey
      tiers: backend
  replicas: 1
  template:
    metadata:
      labels:
        app: pwsurvey
        tiers: backend
    spec:
      containers:
      - image: mariadb
        name: pwsurvey-db
        env:
        - name: MYSQL_DATABASE
          value: pwsurvey
        - name: MYSQL_USER
          value: pwsurvey
        - name: MYSQL_ROOT_PASSWORD
          value: password
        - name: MYSQL_PASSWORD
          value: password
        ports:
        - containerPort: 3306
          name: mysql
          
        volumeMounts:
        - name: pwsurvey-db
          mountPath: /var/lib/mysql
        - name: pwsurvey-log
          mountPath: /var/log/mysql
        - name: pwsurvey-etc
          mountPath: /etc/mysql
          
      volumes:
      - name: pwsurvey-db
        hostPath:
          type: DirectoryOrCreate
          path: /mnt/pwsurvey-db-temp
      - name: pwsurvey-log
        hostPath:
          type: DirectoryOrCreate
          path: /mnt/pwsurvey-log-temp
      - name: pwsurvey-etc
        hostPath:
          type: DirectoryOrCreate
          path: /mnt/pwsurvey-etc-temp
EOF

deployment.apps/pwsurvey-db created


In [96]:
cat << 'EOF' | kubectl apply -f -
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: pwsurvey
spec:
  selector:
    matchLabels:
      app: pwsurvey
      tiers: frontend
  replicas: 1
  template:
    metadata:
      labels:
        app: pwsurvey
        tiers: frontend
    spec:
      containers:
      - image: fjudith/limesurvey
        name: pwsurvey
        env:
        - name: MYSQL_DATABASE
          value: pwsurvey-db
        - name: MYSQL_ROOT_PASSWORD
          value: password
        - name: MYSQL_USER
          value: pwsurvey
        - name: MYSQL_PASSWORD
          value: password
        ports:
        - containerPort: 80
          name: http
          
#        volumeMounts:
#        - name: pwsurvey-appl
#          mountPath: /application/config

          
#      volumes:
#      - name: pwsurvey-appl
#        hostPath:
#          type: DirectoryOrCreate
#          path: /mnt/pwsurvey-appl-temp


EOF

deployment.apps/pwsurvey created


In [61]:
kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
pwsurvey-5d596bccd6-zv6mr   1/1     Running   0          34s


## Services

<img src="media/services.png" alt="SC" width="700" height="700" style="float:left">

In [97]:
cat << 'EOF' | kubectl apply -f -

---
apiVersion: v1
kind: Service
metadata:
  labels:
    app: pwsurvey
    tiers: backend
  name: pwsurvey-db
spec:
  ports:
  - name: mysql
    port: 3306
    targetPort: 3306
    protocol: TCP
  selector:
    app: pwsurvey
    tiers: backend
  type: NodePort
  
---
apiVersion: v1
kind: Service
metadata:
  labels:
    app: pwsurvey
    tiers: frontend
  name: pwsurvey
spec:
  ports:
  - name: http
    port: 80
    targetPort: 80
    protocol: TCP
  selector:
    app: pwsurvey
    tiers: frontend
  type: NodePort

EOF

service/pwsurvey-db created
service/pwsurvey created


In [98]:
kubectl get svc

NAME          TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)          AGE
kubernetes    ClusterIP   10.0.0.1      <none>        443/TCP          3d10h
pwsurvey      NodePort    10.0.63.199   <none>        80:30395/TCP     4s
pwsurvey-db   NodePort    10.0.83.62    <none>        3306:30116/TCP   4s


# Helm

<img src="media/helm.png" alt="SC" width="700" height="700" style="float:left">

# Ingress 
<img src="media/ingress.png" alt="SC" width="700" height="700" style="float:left">

In [99]:
# Create a namespace for your ingress resources
kubectl create namespace ingress-basic

# Add the official stable repo
helm repo add stable https://kubernetes-charts.storage.googleapis.com/

# Use Helm to deploy an NGINX ingress controller
helm install nginx stable/nginx-ingress \
    --namespace ingress-basic \
    --set controller.replicaCount=2 \
    --set controller.nodeSelector."beta\.kubernetes\.io/os"=linux \
    --set defaultBackend.nodeSelector."beta\.kubernetes\.io/os"=linux

Error from server (AlreadyExists): namespaces "ingress-basic" already exists
"stable" has been added to your repositories
Error: cannot re-use a name that is still in use


: 1

In [100]:
kubectl get svc -n ingress-basic

NAME                                  TYPE           CLUSTER-IP    EXTERNAL-IP      PORT(S)                      AGE
cert-manager                          ClusterIP      10.0.0.31     <none>           9402/TCP                     111m
cert-manager-webhook                  ClusterIP      10.0.127.74   <none>           443/TCP                      111m
nginx-nginx-ingress-controller        LoadBalancer   10.0.89.24    52.156.205.223   80:31130/TCP,443:31685/TCP   113m
nginx-nginx-ingress-default-backend   ClusterIP      10.0.17.221   <none>           80/TCP                       113m


<img src="media/certmanager.png" alt="SC" width="700" height="700" style="float:left">

**IMPORTANT - This take at least 2,5 min. to spinup the webhook, so do not hesitate to run it upfront and explain meanwhile.**

In [101]:
# Install the CustomResourceDefinition resources separately
kubectl apply --validate=false -f https://github.com/jetstack/cert-manager/releases/download/v0.14.2/cert-manager.crds.yaml

# Label the ingress-basic namespace to disable resource validation
kubectl label namespace ingress-basic cert-manager.io/disable-validation=true

# Add the Jetstack Helm repository
helm repo add jetstack https://charts.jetstack.io

# Update your local Helm chart repository cache
helm repo update

# Install the cert-manager Helm chart
helm install \
  cert-manager \
  --namespace ingress-basic \
  --version v0.14.2 \
  jetstack/cert-manager

customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io unchanged
customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io unchanged
error: 'cert-manager.io/disable-validation' already has a value (true), and --overwrite is false
"jetstack" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "oteemo" chart repository
...Successfully got an update from the "halkeye" chart repository
...Successfully got an update from the "jetstack" chart repository
...Successfully got an update from the "hpe-storage" chart repository
...S

: 1

In [102]:
kubectl get pods -n ingress-basic

NAME                                                   READY   STATUS    RESTARTS   AGE
cert-manager-776b547ccd-8n44n                          1/1     Running   0          112m
cert-manager-cainjector-95c885477-5jnkm                1/1     Running   0          112m
cert-manager-webhook-6ff9487489-b7tsb                  1/1     Running   0          112m
nginx-nginx-ingress-controller-74bf9bd9f5-lrzgh        1/1     Running   0          114m
nginx-nginx-ingress-controller-74bf9bd9f5-rnnxr        1/1     Running   0          114m
nginx-nginx-ingress-default-backend-54d7956d54-pnbck   1/1     Running   0          114m


In [103]:
cat << 'EOF' | kubectl apply -f -
apiVersion: cert-manager.io/v1alpha2
kind: ClusterIssuer
metadata:
  name: letsencrypt
spec:
  acme:
    server: https://acme-v02.api.letsencrypt.org/directory
    email: dderichsweiler@hotmail.com
    privateKeySecretRef:
      name: letsencrypt
    solvers:
    - http01:
        ingress:
          class: nginx
EOF

clusterissuer.cert-manager.io/letsencrypt created


<img src="media/ingress.png" alt="SC" width="700" height="700" style="float:left">

In [104]:
cat << 'EOF' | kubectl apply -f -
---
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: pw.myk8s.de
  annotations:
    kubernetes.io/ingress.class: nginx  
    cert-manager.io/cluster-issuer: letsencrypt

spec:
  tls:
  - hosts:
    - powerweek.myk8s.de
    secretName: powerweek.myk8s.de
  rules:
  - host: powerweek.myk8s.de
    http:
      paths:
      - path: /
        backend:
          serviceName: pwsurvey
          servicePort: 80
EOF

ingress.extensions/pw.myk8s.de created


In [115]:
kubectl get svc

NAME          TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)          AGE
kubernetes    ClusterIP   10.0.0.1      <none>        443/TCP          3d10h
pwsurvey      NodePort    10.0.63.199   <none>        80:30395/TCP     4m50s
pwsurvey-db   NodePort    10.0.83.62    <none>        3306:30116/TCP   4m50s


In [116]:
kubectl get ingress

NAME          HOSTS                ADDRESS                 PORTS     AGE
pw.myk8s.de   powerweek.myk8s.de   10.240.0.4,10.240.0.5   80, 443   3m15s


In [113]:
kubectl get pods

NAME                           READY   STATUS    RESTARTS   AGE
pwsurvey-5d596bccd6-tjdtd      1/1     Running   0          3m56s
pwsurvey-db-6d6c7965c7-gr7gf   1/1     Running   0          4m1s


In [35]:
kubectl describe ingress pw.myk8s.de

Name:             pw.myk8s.de
Namespace:        default
Address:          10.240.0.4,10.240.0.5
Default backend:  default-http-backend:80 (<none>)
TLS:
  powerweek.myk8s.de terminates powerweek.myk8s.de
Rules:
  Host                Path  Backends
  ----                ----  --------
  powerweek.myk8s.de  
                      /   pwsurvey:80 (<none>)
Annotations:
  kubernetes.io/ingress.class:                       nginx
  cert-manager.io/cluster-issuer:                    letsencrypt
  kubectl.kubernetes.io/last-applied-configuration:  {"apiVersion":"extensions/v1beta1","kind":"Ingress","metadata":{"annotations":{"cert-manager.io/cluster-issuer":"letsencrypt","kubernetes.io/ingress.class":"nginx"},"name":"pw.myk8s.de","namespace":"default"},"spec":{"rules":[{"host":"powerweek.myk8s.de","http":{"paths":[{"backend":{"serviceName":"pwsurvey","servicePort":80},"path":"/"}]}}],"tls":[{"hosts":["powerweek.myk8s.de"],"secretName":"powerweek.myk8s.de"}]}}

Events:
  Type    Reason  Age   Fro

In [ ]:
kubectl get svc

<img src="media/secret.png" alt="SC" width="700" height="700" style="float:left">

In [117]:
kubectl get secret

NAME                  TYPE                                  DATA   AGE
default-token-g8gzp   kubernetes.io/service-account-token   3      3d10h
powerweek.myk8s.de    kubernetes.io/tls                     3      71m
pw.myk8s.de           kubernetes.io/tls                     3      131m


In [ ]:
kubectl describe secret pw.myk8s.de

In [ ]:
kubectl get svc -A

# delete demo

In [118]:
cat << 'EOF' | kubectl delete -f -
---
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: pw.myk8s.de
  annotations:
    kubernetes.io/ingress.class: nginx  
    cert-manager.io/cluster-issuer: letsencrypt

spec:
  tls:
  - hosts:
    - pw.myk8s.de
    secretName: pw.myk8s.de
  rules:
  - host: pw.myk8s.de
    http:
      paths:
      - path: /
        backend:
          serviceName: pwsurvey
          servicePort: 80
          
---
apiVersion: cert-manager.io/v1alpha2
kind: ClusterIssuer
metadata:
  name: letsencrypt
spec:
  acme:
    server: https://acme-v02.api.letsencrypt.org/directory
    email: dderichsweiler@hotmail.com
    privateKeySecretRef:
      name: letsencrypt
    solvers:
    - http01:
        ingress:
          class: nginx
          
---
apiVersion: v1
kind: Service
metadata:
  labels:
    app: pwsurvey
    tiers: backend
  name: pwsurvey-db
spec:
  ports:
  - name: mysql
    port: 3306
    targetPort: 3306
    protocol: TCP
  selector:
    app: pwsurvey
    tiers: backend
  type: NodePort
  
---
apiVersion: v1
kind: Service
metadata:
  labels:
    app: pwsurvey
    tiers: frontend
  name: pwsurvey
spec:
  ports:
  - name: http
    port: 80
    targetPort: 80
    protocol: TCP
  selector:
    app: pwsurvey
    tiers: frontend
  type: NodePort
  
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: pwsurvey-db
spec:
  selector:
    matchLabels:
      app: pwsurvey
      tiers: backend
  replicas: 1
  template:
    metadata:
      labels:
        app: pwsurvey
        tiers: backend
    spec:
      containers:
      - image: mariadb
        name: pwsurvey-db
        env:
        - name: MYSQL_DATABASE
          value: pwsurvey
        - name: MYSQL_USER
          value: pwsurvey
        - name: MYSQL_ROOT_PASSWORD
          value: password
        - name: MYSQL_PASSWORD
          value: password
        ports:
        - containerPort: 3306
          name: mysql

---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: pwsurvey
spec:
  selector:
    matchLabels:
      app: pwsurvey
      tiers: frontend
  replicas: 1
  template:
    metadata:
      labels:
        app: pwsurvey
        tiers: frontend
    spec:
      containers:
      - image: fjudith/limesurvey
        name: pwsurvey
        env:
        - name: MYSQL_DATABASE
          value: pwsurvey-db
        - name: MYSQL_ROOT_PASSWORD
          value: password
        - name: MYSQL_USER
          value: pwsurvey
        - name: MYSQL_PASSWORD
          value: password
        ports:
        - containerPort: 80
          name: http     
EOF

ingress.extensions "pw.myk8s.de" deleted
clusterissuer.cert-manager.io "letsencrypt" deleted
service "pwsurvey-db" deleted
service "pwsurvey" deleted
deployment.apps "pwsurvey-db" deleted
deployment.apps "pwsurvey" deleted


In [ ]:
helm uninstall -n ingress-basic nginx
helm uninstall -n ingress-basic cert-manager

In [ ]:
kubectl delete ns ingress-basic